In [101]:
import ujson
import re
import pandas as pd

from tqdm import tqdm_notebook
from glob import glob
from gensim.models.word2vec import Word2Vec
from itertools import product
from wordfreq import top_n_list
from IPython.display import Markdown, HTML, display
from tabulate import tabulate

In [117]:
pd.options.display.max_rows = 1000

In [18]:
def read_tweets(pattern):
    
    tweets = []
    for path in glob(pattern):
        with open(path) as fh:
            for line in tqdm_notebook(fh):
                
                tweet = ujson.loads(line)
                
                text = re.sub('(#|@|http)\S+', '', tweet['body'])
                tokens = re.findall('[a-z0-9]+', text.lower())
                
                tweets.append(tokens)
                
    return tweets

In [19]:
rural = read_tweets('../../data/geo-lt10k.json/part-0000*')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [15]:
urban = read_tweets('../../data/geo-gt1m.json/part-0000*')

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [20]:
rural_model = Word2Vec(rural, workers=8)

In [22]:
urban_model = Word2Vec(urban, workers=8)

In [21]:
rural_model.most_similar('earth')

[('planet', 0.7829978466033936),
 ('dwell', 0.5269863605499268),
 ('world', 0.5174082517623901),
 ('universe', 0.5011920928955078),
 ('purpose', 0.4989180564880371),
 ('depending', 0.49261489510536194),
 ('mankind', 0.48292750120162964),
 ('christ', 0.47708719968795776),
 ('visible', 0.46374422311782837),
 ('species', 0.4624600410461426)]

In [23]:
urban_model.most_similar('earth')

[('planet', 0.8616721034049988),
 ('world', 0.5897253751754761),
 ('side', 0.5761807560920715),
 ('ocean', 0.574997067451477),
 ('purpose', 0.5739495158195496),
 ('soil', 0.5735812187194824),
 ('darkness', 0.5720939636230469),
 ('dwell', 0.5691649913787842),
 ('surface', 0.5686110854148865),
 ('path', 0.5609077215194702)]

In [52]:
def score_shift(m1, m2, token, n=10):
    
    m1_similar = [t for t, _ in m1.most_similar(token, topn=n)]
    m2_similar = [t for t, _ in m2.most_similar(token, topn=n)]
    
    score = 0
    for t1, t2 in product(m1_similar, m2_similar):
        score += m2.similarity(t1, t2)
        
    return score

In [53]:
score_shift(urban_model, rural_model, 'fuck')

49.193618851642924

In [126]:
data = []

for token in top_n_list('en', 5000):
    try:
        score = score_shift(urban_model, rural_model, token)
        data.append((token, score))
    except: pass

In [127]:
df = pd.DataFrame(data, columns=('token', 'score'))

In [146]:
df.sort_values('score').head(300)

,token,score
1048,lady,-5.571520
3903,stranger,0.839832
2005,ma,1.924596
1824,sat,3.972938
2629,drew,6.056310
2960,occasionally,7.195861
3504,sessions,8.395590
1073,russian,8.973371
2321,bush,9.178413
3334,genuine,9.217894


In [139]:
def compare(token):
    
    u_similar = urban_model.most_similar(token)
    r_similar = rural_model.most_similar(token)
    
    u_tokens = [[t] for t, _ in u_similar]
    r_tokens = [[t] for t, _ in r_similar]
    
    return u_tokens, r_tokens

In [147]:
for token in df.sort_values('score').head(300)['token']:
    u, r = compare(token)
    display(Markdown(f'# {token}'))
    display(Markdown(f'#### Urban'))
    print(tabulate(u))
    display(Markdown(f'#### Rural'))
    print(tabulate(r))

# lady

#### Urban

--------
girl
guy
woman
kid
man
dude
boy
chick
coworker
toddler
--------


#### Rural

--------
knights
mustangs
cougars
panthers
wildcats
tigers
eagles
bulldogs
lions
indians
--------


...

# mid

#### Urban

------
early
3rd
1st
2nd
5th
during
6th
7th
8th
pre
------


#### Rural

---------
recap
brewster
1900
2016
northwest
rises
1993
2022
seventh
nov
---------
